# project path

 - E:\Campus-X\campusx_mlops\campusx_project\mlops-mini-project\notebooks

 - https://github.com/campusx-official/mlops-mini-project/tree/master

In [4]:
%pip install -q mlflow pandas scikit-learn nltk numpy dagshub dvc jupyter

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install jupyter

  Using cached jupyter-1.1.1-py2.py3-none-any.whl.metadata (2.0 kB)
  Using cached notebook-7.4.5-py3-none-any.whl.metadata (10 kB)
  Using cached jupyter_console-6.6.3-py3-none-any.whl.metadata (5.8 kB)
  Using cached nbconvert-7.16.6-py3-none-any.whl.metadata (8.5 kB)
  Using cached ipywidgets-8.1.7-py3-none-any.whl.metadata (2.4 kB)
  Using cached jupyterlab-4.4.7-py3-none-any.whl.metadata (16 kB)
  Using cached widgetsnbextension-4.0.14-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.15-py3-none-any.whl.metadata (20 kB)
  Using cached async_lru-2.0.5-py3-none-any.whl.metadata (4.5 kB)
  Using cached jupyter_lsp-2.3.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached jupyter_server-2.17.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached jupyterlab_server-2.27.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached notebook_shim-0.2.4-py3-none-any.whl.metadata (4.0 kB)
  Using cached argon2_cffi-25.1.0-py3-none-any.whl.metadata (4.1 kB)
  Using cached jupyter_ev

In [1]:
# check the all installed installed version
import dvc, mlflow, pandas, numpy, dagshub, sklearn, jupyter
print("✅ All packages installed successfully!")

✅ All packages installed successfully!


In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\aamir\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\aamir\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv')
df.sample(10)

,tweet_id,sentiment,content
8475,1962195602,relief,@_HarryKim OOC: It's okay. Maybe our schedul...
23813,1694705060,happiness,"woo just made this, follow me"
11258,1963157199,sadness,@arizonaobvious that sucks
10017,1962803421,hate,"damn fight night 4 demo won't load, keeps cras..."
2729,1957614712,love,@BrianLimond but lorraine kelly is a sexy mama!
29145,1751106427,surprise,Off to prom. Say hi to me
35992,1753218064,worry,i had a vivid dream last night! i was crooning...
8209,1962067688,relief,@bethanyshondark Ouch. Better get used to it. ...
6738,1961427598,neutral,@moriagerard I haven't had one for a long time...
21756,1694293402,happiness,@cheergod2002 yeah I can tell ha ha.


In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()          # initialize lemmatizer
    text = text.split()                       # split into words
    text = [lemmatizer.lemmatize(word) for word in text]  # lemmatize each word
    return " ".join(text)                     # join back into string

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))  # load stopwords
    text = [word for word in str(text).split() if word not in stop_words]  # remove stopwords
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])  # keep only non-digits
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()                       # split into words
    text = [word.lower() for word in text]    # convert to lowercase
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)  # remove punctuation
    text = re.sub(r'\s+', ' ', text).strip()  # remove extra spaces
    text = text.replace('؛', "")             # remove special char
    text = re.sub('\s+', ' ', text).strip()  # remove extra spaces
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')  # regex for URLs
    return url_pattern.sub(r'', text)                   # replace URLs with empty string

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)           # step 1: lowercase
        df['content'] = df['content'].apply(remove_stop_words)    # step 2: remove stopwords
        df['content'] = df['content'].apply(removing_numbers)     # step 3: remove numbers
        df['content'] = df['content'].apply(removing_punctuations)# step 4: remove punctuations
        df['content'] = df['content'].apply(removing_urls)        # step 5: remove URLs
        df['content'] = df['content'].apply(lemmatization)        # step 6: lemmatization
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')            # print error if any
        raise



<>:33: SyntaxWarning: invalid escape sequence '\s'
<>:33: SyntaxWarning: invalid escape sequence '\s'
C:\Users\aamir\AppData\Local\Temp\ipykernel_2956\2725294146.py:33: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()  # remove extra spaces


In [6]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [7]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [8]:
# create a boolean mask where sentiment is either 'happiness' or 'sadness'
x = df['sentiment'].isin(['happiness', 'sadness'])

# filter the DataFrame using the mask (keep only happiness and sadness rows)
df = df[x]

In [9]:
# replace sentiment labels with numeric values (sadness → 0, happiness → 1)
df['sentiment'] = df['sentiment'].replace({'sadness': 0, 'happiness': 1})

# display the first 5 rows of the DataFrame
df.head(5)


C:\Users\aamir\AppData\Local\Temp\ipykernel_2956\4269443875.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness': 0, 'happiness': 1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [10]:
df.sample(10)

,sentiment,content
21417,1,anyone need help image let know convo forum s ...
27129,0,smell like beach job
24945,0,sore yesterday definitely feeling hill now leg...
21302,1,forgot lumpia pancit fridge last night yay
15241,0,missing best friend commme back kayla going li...
4301,1,back home great time
26567,1,diannemr like that
24582,1,macmuso thank much glad like them
4912,0,well only one week left holiday sad sad
10628,0,ayy fml nothing perfect


In [11]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
import dagshub

# https://dagshub.com/aamir490/mlops-mini-project.mlflow
mlflow.set_tracking_uri('https://dagshub.com/aamir490/mlops-mini-project.mlflow')
dagshub.init(repo_owner='aamir490', repo_name='mlops-mini-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")
#mlflow.set_experiment("Logistic Regression Baseline v2")

# https://dagshub.com/aamir490/mlops-mini-project.mlflow


Accessing as aamir490

Initialized MLflow to track repo "aamir490/mlops-mini-project"

Repository aamir490/mlops-mini-project initialized!

2025/09/10 08:05:35 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/b5379880dab44d6c976de7b741d42aef', creation_time=1757471735135, experiment_id='1', last_update_time=1757471735135, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

# Now our experiment start :-

In [13]:
import shutil
import os
# Remove the old folder if it exists
if os.path.exists("saved_model"):
    shutil.rmtree("saved_model")

# Now save safely
mlflow.sklearn.save_model(model, "saved_model")
mlflow.log_artifact("saved_model")


In [15]:
# import os
# import mlflow
# import joblib
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# # Start an MLflow run
# with mlflow.start_run():

#     # -----------------------------
#     # Log preprocessing parameters
#     # -----------------------------
#     mlflow.log_param("vectorizer", "Bag of Words")
#     mlflow.log_param("num_features", 1000)
#     mlflow.log_param("test_size", 0.2)

#     # -----------------------------
#     # Model building and training
#     # -----------------------------
#     model = LogisticRegression()
#     model.fit(X_train, y_train)

#     # Log model type
#     mlflow.log_param("model", "Logistic Regression")

#     # -----------------------------
#     # Model evaluation
#     # -----------------------------
#     y_pred = model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred)
#     recall = recall_score(y_test, y_pred)
#     f1 = f1_score(y_test, y_pred)

#     # Log metrics
#     mlflow.log_metric("accuracy", accuracy)
#     mlflow.log_metric("precision", precision)
#     mlflow.log_metric("recall", recall)
#     mlflow.log_metric("f1_score", f1)

#     # -----------------------------
#     # Log the trained model as artifact
#     # -----------------------------
#     model_path = "logreg_model.pkl"
#     joblib.dump(model, model_path)
#     mlflow.log_artifact(model_path, artifact_path="models")  # DagsHub-compatible

#     # -----------------------------
#     # Save and log the notebook
#     # -----------------------------
#     notebook_path = r"E:\Campus-X\campusx_mlops\campusx_projectt\mlops-mini-project\notebooks\exp1_baseline_model (1).ipynb"
#     os.system(f"jupyter nbconvert --to notebook --execute --inplace \"{notebook_path}\"")
#     mlflow.log_artifact(notebook_path)

#     # -----------------------------
#     # Print results
#     # -----------------------------
#     print(f"Accuracy: {accuracy}")
#     print(f"Precision: {precision}")
#     print(f"Recall: {recall}")
#     print(f"F1 Score: {f1}")
#     print("✅ Model + metrics logged successfully 🚀")

    

# # code 2    
import os
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# ✅ Safety: end any previous active run
mlflow.end_run()

with mlflow.start_run():
    # Log preprocessing params
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)

    # Train model
    model = LogisticRegression()
    model.fit(X_train, y_train)

    # Log model params
    mlflow.log_param("model", "Logistic Regression")

    # Evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    # Log metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)

    # ✅ Save + log model as artifact
    model_dir = "saved_model"
    if os.path.exists(model_dir):
        import shutil
        shutil.rmtree(model_dir)  # clear old folder if exists

    mlflow.sklearn.save_model(model, model_dir)
    mlflow.log_artifact(model_dir)

    # Log the notebook
    notebook_path = r"E:\Campus-X\campusx_mlops\campusx_projectt\mlops-mini-project\notebooks\exp1_baseline_model (1).ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace \"{notebook_path}\"")
    mlflow.log_artifact(notebook_path)

    # Print results for quick check
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print("✅ Model + metrics logged successfully 🚀")


Accuracy: 0.7769
Precision: 0.7690
Recall: 0.7773
F1 Score: 0.7732
✅ Model + metrics logged successfully 🚀
